# Introduction

This tutorial aims to elucidate the information stored with a SAM and BAM files, and how such files can be read, or parsed, within the Python programming language and on the command line.

The goals from this tutorial include understanding:

- What is a SAM/BAM file?
- How are they created?
- How can we manipulate them?


The tutorial includes a small E.coli dataset (100x) in the form of a compressed fastq file to demonstrate the key learning objectives. This colab was generated using information from https://labs.epi2me.io/notebooks/Introduction_to_SAM_and_BAM_files.html

## SAM and BAM

A SAM file (usually named *.sam) is used to represent aligned sequences. It is particularly useful for storing the results of aligning genomic or transcriptomic sequence reads aligned against a reference genome sequence. The BAM file format is a compressed form of SAM. This has the disadvantage that it is not readable by a human but has the advantage of being smaller than the corresponding SAM file and thus easier to share and copy between locations.

You can read about SAM and BAM formats here:
 - Li, H., Handsaker, B., Wysoker, A., Fennell, T., Ruan, J., Homer, N., Marth, G., Abecasis, G., Durbin, R., & 1000 Genome Project Data Processing Subgroup (2009). The Sequence Alignment/Map format and SAMtools. *Bioinformatics*, **25**, 2078–2079. https://doi.org/10.1093/bioinformatics/btp352 and
-  [https://samtools.github.io/hts-specs/SAMv1.pdf](https://samtools.github.io/hts-specs/SAMv1.pdf).

We can view BAM files graphically using a specialised genome browser software such as:
- [IGV](https://igv.org/)
- [Tablet](https://ics.hutton.ac.uk/tablet/)
- [Artemis / BAMview](http://sanger-pathogens.github.io/Artemis/BamView/) 

#Getting started

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
!pip install epi2melabs
from epi2melabs import ping
tutorial_name = "bam_tutorial"
pinger = ping.Pingu()
pinger.send_notebook_ping('start', tutorial_name)

#### Create create and set a working directory

In [5]:

working_dir = '/epi2melabs/{}/'.format(tutorial_name)
!mkdir -p "$working_dir"
%cd "$working_dir"

/epi2melabs/bam_tutorial


###Sample Data
For the purpose of an introduction to the SAM/BAM format, we have uploaded a small E.coli dataset (100x) which is available to download here in the form of a compressed fastq.

To download the sample file we run the linux command wget.

In [ ]:
bucket = "ont-exd-int-s3-euwst1-epi2me-labs"
domain = "s3-eu-west-1.amazonaws.com"
site = "https://{}.{}".format(bucket, domain)

# E. coli sample dataset
!wget -O 100X_ecoli_zymo_r103.fastq.gz \
    "$site/$tutorial_name/100X_ecoli_zymo_r103.fastq.gz"

# Reference sequence
!wget -O ecoli_k12.fasta \
    "$site/$tutorial_name/ecoli_k12.fasta"

Before we move on, let's quickly inspect the sample .FASTQ we have downloaded using seqkit. This should give us an overview which we can refer back to later. Again, click play to run the following command.

In [ ]:
!conda install -c bioconda seqkit

In [8]:
!seqkit stats *.fastq.gz

file                           format  type  num_seqs      sum_len  min_len  avg_len  max_len
100X_ecoli_zymo_r103.fastq.gz  FASTQ   DNA     52,061  448,038,968      103    8,606  111,547


##Aligning with Minimap2
In this demonstration we use minimap2, the aligner most commonly used with ONT data. It is super fast, accurate and easy to use. Minimap2 lives here on github, and the README there provides detailed usage instructions.

Let's break down the basic invocation of the tool:

>minimap2 -a -x map-ont ref.fa query.fq > alignment.sam

minimap2 is the base command.

-a is a flag which turns on SAM output, which is what we want, otherwise minimap2 outputs an alternative PAF format

-x map-ont tells minimap2 that we want to align ONT data, allowing the tool to modify its internal parameters to make the best alignments
ref.fa is a positional argument, which means it must come first after any flags such as -a. This term refers to the reference, the sequence(s) against which your reads will be aligned.
query.fq represents the .FASTQ file containing your reads

alignment.sam is a statement saying 'redirect the output to a file called alignment.sam'. Don't worry, alignment.sam will be created if it doesn't already exist.

Substituting that example invocation for our own, let's map the sample data downloaded above against the E. coli K12 reference. Run the cell as before:

In [ ]:
!conda install -c bioconda minimap2

In [ ]:
reference = 'ecoli_k12.fasta' 
query = '100X_ecoli_zymo_r103.fastq.gz'

!minimap2 -ax map-ont $reference $query > alignment.sam

##Compressing, sorting, indexing with Samtools
We should now have a SAM file. We can do quite a lot with it already, but in general we want to compress the file to make best use of disk space, and because many tools (such as IGV) require BAM files as input. As such, its always good practice to convert SAM to BAM.

Let's introduce samtools to take care of that for us. Here's the command:

> samtools view -S -b alignment.sam > alignment.bam

samtools is the base command.
view is the subcommand.

 Samtools has many different sub-commands. This one, view, is used for converting SAM/BAM files between formats, as well as accessing certain records within the file, e.g. those aligned to a specific chromosome.

-S is a flag that tells samtools view that our input will be a SAM file.

-b is a flag that tells samtools view that it should output BAM records.

alignment.sam is the SAM file we made, which we inputting to this command.

alignment.bam is the file to which we will redirect the BAM output.

For every samtools subcommand, there are many flags and arguments that you may provide to customise its behaviour. Whenever you need to do something, but don't immediately know how to, you should consult the documentation.

There a few steps we want to perform during our conversion from SAM to BAM, as follows:

- Compression - this step reduces the size of the SAM file.
- Sorting - this step sorts the records by position on the reference, rather than by the original order that the reads in the .FASTQ file had. This makes it much quicker for downstream tools to access data for reads co-located on the reference sequence.
- Indexing - this step creates an extra file (.bai or BAM index) allowing programs to find reads spanning a genomic location without having to read the whole file.

In fact it is possible to compress and sort a BAM file in a single command:

In [ ]:
!conda install -c bioconda samtools

In [14]:
!samtools sort -o sorted.alignment.bam alignment.sam

In [15]:
!samtools index sorted.alignment.bam

#Get a summary of the records in the file
As you can see from the output of flagstat below, we have a few thousand secondary and supplementary alignments each, and a mapping rate of 93.98%. The remainder, i.e. the total number of records minus the mapped reads, 65646 - 61694, should give us the number of unmapped reads.

In [ ]:
!samtools flagstat sorted.alignment.bam

##Reading our BAM file in Python with pysam
Sometimes, one needs more powerful programmatic access than that which is available via command line tools alone. The good news is that there are several ways to access SAM and BAM files using Python and other languages.

For example, pysam is a python library which makes the manipulation of various biological formats as straightforward as possible. Specifically, Pysam provides the pysam.AlignmentFile parser.

We can use the parser to iterate over the alignments within our BAM file, and quite easily visualise the alignment results, as below:

In [ ]:
!pip install pysam
import pysam
import numpy as np

read_lengths = []
with pysam.AlignmentFile("sorted.alignment.bam", "rb") as bam:
    for read in bam.fetch('U00096.3'):
        read_lengths.append(read.query_length)
print("Mean read length: {:.0f} bases.".format(np.mean(read_lengths)))

To create a simple tabular summary of per-read statistics the stats_from_bam program in pomoxis is useful:

In [ ]:
# run the alignment summarizer program
!mamba install -c bioconda pomoxis

In [ ]:
!stats_from_bam sorted.alignment.bam > sorted.alignment.bam.stats

In [26]:
!pip install aplanat
import pandas as pd
import aplanat
from aplanat.hist import histogram
from bokeh.layouts import gridplot
df = pd.read_csv("sorted.alignment.bam.stats", sep="\t")

p1 = histogram(
    [df['read_length']], title="Read lengths",
    x_axis_label="read length / bases", y_axis_label="count")
p1.xaxis.formatter.use_scientific = False
p2 = histogram(
    [df['acc']], title="Read accuracy",
    x_axis_label="% accuracy", y_axis_label="count")
aplanat.show(gridplot((p1, p2), ncols=2))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
